In [1]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [2]:
quarterly = pd.read_sql(
    """
    select datekey, reportperiod, ticker, netinc, equity
    from sf1
    where dimension='ARQ' and equity>0
    order by ticker, datekey
    """,
    conn
)
quarterly = quarterly.dropna()

In [3]:
quarterly["equitylag"] = quarterly.groupby("ticker").equity.shift()
quarterly["roe"] = quarterly.netinc / quarterly.equitylag

# save last report for each firm
quarterly = quarterly.groupby("ticker").last()
quarterly = quarterly[quarterly.reportperiod.astype(str)>="2022-06-01"]

# drop variables other than roe and ticker (ticker=index)
quarterly = quarterly[["roe"]]

In [4]:
daily1 = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap
    from daily 
    where date>='2023-01-01' and pb>0
    order by ticker, date
    """,
    conn
)
daily1 = daily1.dropna()
daily1 = daily1.groupby("ticker").last()

In [5]:
daily2 = pd.read_sql(
    """ 
    select date, ticker, close_ as price
    from sep 
    where date>='2023-01-01'
    order by ticker, date
    """,
    conn
)
daily2 = daily2.dropna()
daily2 = daily2.groupby("ticker").last()

In [6]:
df = pd.concat((quarterly, daily1, daily2), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3609 entries, A to YTPG
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   roe        3354 non-null   float64
 1   date       3285 non-null   object 
 2   pb         3285 non-null   float64
 3   marketcap  3285 non-null   float64
 4   date       3544 non-null   object 
 5   price      3544 non-null   float64
dtypes: float64(4), object(2)
memory usage: 197.4+ KB


In [7]:
df = df[df.price>5]
df.shape

(2621, 6)

In [8]:
df["size_rank"] = df.marketcap.rank(ascending=False)
df = df[df.size_rank > 500]

In [9]:
filter1 = df.roe >= df.roe.quantile(0.7)
filter2 = df.pb <= df.pb.quantile(0.3)
df = df[filter1 & filter2].copy()
df.to_csv("files/first_portfolio.csv")